In [2]:
import pandas as pd
import numpy as np
from scipy import spatial
from geopy.distance import vincenty
from shapely.geometry import Point
from shapely.geometry.polygon import LinearRing

In [3]:
bbl_data = pd.read_csv('/Users/Laura/GoogleDrive/NYU/Capstone/NYCOpenData/Data/bbl_lat_long_zipcode.csv')
bbl_data['bbl'] = bbl_data['bbl'].astype(int)
bbl_data['zipcode'] = bbl_data['zipcode'].astype(int)
bbl_data = bbl_data.drop_duplicates().reset_index(drop=True)
bbl_data = bbl_data[:100]
bbl_data.head()

,bbl,latitude,longitude,zipcode
0,1000010010,40.688799,-74.018675,10004
1,1000057501,40.702266,-74.010607,10004
2,1000070038,40.703440,-74.011087,10004
3,1000110012,40.704815,-74.012122,10004
4,1000110014,40.704813,-74.011989,10004


In [8]:
def fix_geom(data):
    if 'the_geom' in data:
        data['latitude'] = np.nan * data.index
        data['longitude'] = np.nan * data.index
        for i in data.index:
            temp = str(data['the_geom'][i])
            temp = str.strip(temp, 'POINT (')
            temp = str.strip(temp, ')')
            temp = temp.split()
            try:
                data['latitude'][i] = temp[1]
                data['longitude'][i] = temp[0]
            except: 
                pass
    elif 'Latitude' in data:
        data = data.rename(index=str, columns={"Latitude": "latitude", "Longitude": "longitude"})
    elif 'LATITUDE' in data:
        data = data.rename(index=str, columns={"LATITUDE": "latitude", "LONGITUDE": "longitude"})
    else:
        #print ("hmm, something wrong")
        pass
    return data

In [28]:
def bbl_dist_to(bbl_data,data,col_name):
    
    data = data[['latitude', 'longitude']]
    data = data.convert_objects(convert_numeric=True)
    data_loc_matrix = data.values.tolist()
    
    dist_list = []
    for i in bbl_data.index:
        #print i,
        try:
            bbl_coord = (bbl_data['latitude'][i],bbl_data['longitude'][i])
            data_coord = data_loc_matrix[spatial.KDTree(data_loc_matrix).query(bbl_coord)[1]]
            distance = vincenty(bbl_coord, data_coord).miles
        except: 
            distance = np.nan
        dist_list = dist_list + [distance]
    bbl_data[col_name] = dist_list 

In [67]:
files_to_read = [  
#    '01_botanical_gardens.csv',
#    '02_path_stations.csv',
#    '03_libraries.csv',
#    '04_colleges.csv',
#    '05_sidewalk_cafes.csv',
#    '06_day_care_centers.csv',
#    '07_dsny_special_waste.csv',
#    '08_link_nyc_centers.csv',
#    '09_beacon_centers.csv',
#    '10_out_of_school_youth_centers.csv',
#    '11_runaway_homeless_youth_centers.csv',
#    '12_agency_service_centers.csv',
#    '13_summer_youth_employment_centers.csv',
#    '14_adult_continuing_ed_centers.csv',
#    '15_young_adult_internship_centers.csv',
#    '16_after_school_programs.csv',
#    '17_wifi_hotpots.csv',
#    '18_public_pay_phone.csv',
#    '19_museums.csv',
#    '20_theaters.csv',
#    '21_art_galleries.csv',
#    '22_hurricane_evacuation_centers.csv',
#    '23_airports.csv']
#    '24_spray_showers.csv',
#    '25_post_offices.csv',
#    '26_public_recycling_bins.csv'] # something not right
#    '27_universial_pre_k.csv',
#    '28_community_facilities.csv'
    '29_flooding.csv',
    '30_golf_courses.csv',
    '31_greenthumb.csv',
    '32_landfills.csv',
    #'33_landmarks.csv',
    #'34_park_properties.csv',
    '35_parking_lots.csv',
    #'36_playareas.csv',
    #'37_plazas.csv',
    '38_pools.csv',
    #'39_sandy.csv',
    #'40_schoolyards_playgrounds.csv',
    '41_skateparks.csv',
    '42_swimming_pools.csv',
    '43_waterfronts.csv',
    '44_beaches.csv',
    '45_boardwalks.csv',
    '46_business_improvement_districts.csv'
]

In [68]:
for file in files_to_read:
    print (file)
    feature_name =  file[3:-4] + '_dist'    
    data = pd.read_csv('Data/'+ file)
    data = fix_geom(data)
    bbl_dist_to(bbl_data,data,feature_name)
bbl_data.head()

29_flooding.csv


/Users/Laura/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


30_golf_courses.csv
31_greenthumb.csv
32_landfills.csv
35_parking_lots.csv
38_pools.csv
41_skateparks.csv
42_swimming_pools.csv


,bbl,latitude,longitude,zipcode,food_stamp_centers_zipcode_count,hs_alternative_centers_zipcode_count,job_centers_zipcode_count,farmers_markets_boro_count,nature_preserves_boro_count,flooding_dist,golf_courses_dist,greenthumb_dist,landfills_dist,parking_lots_dist,polling_places_zipcode_count,pools_dist,senior_centers_zipcode_count,skateparks_dist,swimming_pools_dist
0,1000010010,40.688799,-74.018675,10004,0,0,0,39,4,0.125125,5.372576,1.472068,8.350589,0.938130,0,1.388142,0,4.194805,1.126268
1,1000057501,40.702266,-74.010607,10004,0,0,0,39,4,0.043302,6.307213,1.570311,8.378575,0.157418,0,1.664007,0,3.412311,0.714007
2,1000070038,40.703440,-74.011087,10004,0,0,0,39,4,0.127677,6.387066,1.804256,8.440703,0.170768,0,1.593081,0,3.408928,0.632119
3,1000110012,40.704815,-74.012122,10004,0,0,0,39,4,0.186154,6.479820,1.719523,8.535380,0.219669,0,1.520167,0,3.431421,0.523042
4,1000110014,40.704813,-74.011989,10004,0,0,0,39,4,0.190945,6.479869,1.718442,8.529238,0.221979,0,1.517976,0,3.424777,0.527177


In [32]:
#bbl_data['flooding_dist'].round().value_counts(dropna=False)

In [37]:
def fix_MULTIPOLYGON_geom(data):
    data['latitude'] = np.nan * data.index
    data['longitude'] = np.nan * data.index
    for i in data.index:
        try:
            temp = data['the_geom'][i]
            temp = temp.replace(',', '')
            temp = str.strip(temp, 'MULTIPOLYGON (((')
            temp = str.strip(temp, ')))')
            temp = map(float, temp.split())
            temp = np.asarray(temp).reshape(len(temp)/2,2)
            temp = LinearRing(temp)
            temp = temp.representative_point().wkt
            temp = str.strip(temp, 'POINT (')
            temp = str.strip(temp, ')')
            temp = temp.split()
            data['latitude'][i] = temp[1]
            data['longitude'][i] = temp[0]
        except:
            data['latitude'][i] = np.nan
            data['longitude'][i] = np.nan
    data = data.dropna(axis=0)
    return data

In [38]:
# flooding
# gold courses
# greenthumb
# landfills
# park properties
# parking_lots
# playareas
# plazas
# pools
# sandy
# schoolyards_playgrounds
# skateparks
# swimming_pools
# waterfronts
# beaches
# boardwalks
# business_improvement_districts center
cols = ['the_geom']
data = pd.read_csv('Data/DistanceToArea/park_properties.csv',usecols=cols)

data = fix_MULTIPOLYGON_geom(data)
data.head()
data=data.drop('the_geom',axis=1)
data.to_csv('Data/DistanceToArea/park_properties.csv',index=False)
data.head()

/Users/Laura/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Laura/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Laura/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Laura/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set o

,latitude,longitude
1,40.844256,-73.783972
2,40.577930,-73.836249
3,40.841715,-73.876411
5,40.581058,-74.004144
6,40.729328,-73.962214


In [14]:
# Boroughs
borough_files_to_read = [  
    '60_farmers_markets.csv',
    '61_nature_preserves.csv']

In [15]:
for file in borough_files_to_read:
    print (file)
    feature_name =  file[3:-4] + '_boro_count' 
    data = pd.read_csv('Data/'+ file)
    data = data.replace(['Manhattan', 'Bronx', 'Brooklyn', 'Queens', 'Staten Island'], [1, 2, 3, 4, 5])
    boro_dict = data['Borough'].value_counts()
    count_list = []
    for i in bbl_data.index:
        boro = int(str(bbl_data['bbl'][i])[0])
        count = boro_dict.get(boro)
        count_list.append(count)
    bbl_data[feature_name] = count_list

bbl_data.head()

60_farmers_markets.csv
61_nature_preserves.csv


,bbl,latitude,longitude,zipcode,food_stamp_centers_zipcode_count,hs_alternative_centers_zipcode_count,job_centers_zipcode_count,farmers_markets_boro_count,nature_preserves_boro_count
0,1000010010,40.688799,-74.018675,10004,0,0,0,39,4
1,1000057501,40.702266,-74.010607,10004,0,0,0,39,4
2,1000070038,40.703440,-74.011087,10004,0,0,0,39,4
3,1000110012,40.704815,-74.012122,10004,0,0,0,39,4
4,1000110014,40.704813,-74.011989,10004,0,0,0,39,4


In [31]:
# Zipcode
zipcode_files_to_read = [  
    '80_food_stamp_centers.csv',
    '81_hs_alternative_centers.csv',
    '82_job_centers.csv',
    '83_polling_places.csv',
#    '84_senior_centers.csv',
#    '85_potholes.csv',
#    '86_restaurant_grades.csv',
    '87_trees.csv']

In [32]:
for file in zipcode_files_to_read:
    print (file)
    feature_name =  file[3:-4] + '_zipcode_count' 
    data = pd.read_csv('Data/'+ file)
    zipcode_dict = data['Zip Code'].value_counts()
    #print zipcode_dict
    count_list = []
    for i in bbl_data.index:
        zipcode = bbl_data['zipcode'][i]
        count = zipcode_dict.get(zipcode)
        if count == None:
            count = 0    
        count_list.append(count)
    bbl_data[feature_name] = count_list
bbl_data.head()

80_food_stamp_centers.csv
81_hs_alternative_centers.csv
82_job_centers.csv
83_polling_places.csv
87_trees.csv


,bbl,latitude,longitude,zipcode,food_stamp_centers_zipcode_count,hs_alternative_centers_zipcode_count,job_centers_zipcode_count,polling_places_zipcode_count,senior_centers_zipcode_count,potholes_zipcode_count,restaurant_grades_zipcode_count,trees_zipcode_count
0,1000010010,40.688799,-74.018675,10004,0,0,0,0,0,654,2212,101
1,1000057501,40.702266,-74.010607,10004,0,0,0,0,0,654,2212,101
2,1000070038,40.703440,-74.011087,10004,0,0,0,0,0,654,2212,101
3,1000110012,40.704815,-74.012122,10004,0,0,0,0,0,654,2212,101
4,1000110014,40.704813,-74.011989,10004,0,0,0,0,0,654,2212,101


In [99]:
# with BBLs 
with_bbls_files_to_read = [  
    '90_graffiti.csv']

In [109]:
for file in with_bbls_files_to_read:
    print (file)
    feature_name =  file[3:-4] + '_count' 
    data = pd.read_csv('Data/'+ file)
    with_bbl_dict = data['BBL'].value_counts()
    count_list = []
    for i in bbl_data.index:
        bbl = bbl_data['bbl'][i]
        count = with_bbl_dict.get(bbl)
        if count == None:
            count = 0    
        count_list.append(count)
    bbl_data[feature_name] = count_list
bbl_data.head()

90_graffiti.csv


,bbl,latitude,longitude,zipcode,graffiti_count
0,1000010010,40.688799,-74.018675,10004,0
1,1000057501,40.702266,-74.010607,10004,0
2,1000070038,40.703440,-74.011087,10004,0
3,1000110012,40.704815,-74.012122,10004,0
4,1000110014,40.704813,-74.011989,10004,0


In [17]:
data.columns.values

array(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD', 'PD_DESC',
       'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'JURIS_DESC', 'BORO_NM',
       'ADDR_PCT_CD', 'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'PARKS_NM',
       'HADEVELOPT', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude',
       'Lat_Lon'], dtype=object)

In [71]:
data['OFNS_DESC'].value_counts()

PETIT LARCENY                           822498
HARRASSMENT 2                           604070
ASSAULT 3 & RELATED OFFENSES            521538
CRIMINAL MISCHIEF & RELATED OF          505774
GRAND LARCENY                           429196
DANGEROUS DRUGS                         348469
OFF. AGNST PUB ORD SENSBLTY &           283065
ROBBERY                                 198772
BURGLARY                                191406
FELONY ASSAULT                          184069
DANGEROUS WEAPONS                       124235
MISCELLANEOUS PENAL LAW                 118721
GRAND LARCENY OF MOTOR VEHICLE          102061
OFFENSES AGAINST PUBLIC ADMINI          100118
INTOXICATED & IMPAIRED DRIVING           73730
CRIMINAL TRESPASS                        66544
VEHICLE AND TRAFFIC LAWS                 59140
THEFT-FRAUD                              56762
SEX CRIMES                               54970
FORGERY                                  49303
FRAUDS                                   32054
POSSESSION OF

In [96]:
new_desc = []
for i in data.index:
    desc = str(data['OFNS_DESC'][i])
    if 'LARCENY' in desc or 'ROBBERY' in desc or 'BURGLARY' in desc:
         new_desc.append('Theft')
    elif 'MURDER' in desc or 'MANSLAUGHTER' in desc:
         new_desc.append('Murder')
    elif 'DRUG' in desc:
         new_desc.append('Drugs')
    elif 'ASSAULT' in desc or 'OFFENSES AGAINST PUBLIC ADMINI' in desc or  'HARRASSMENT' in desc:
         new_desc.append('Assault')
    elif 'WEAPONS' in desc:
         new_desc.append('Weapons')
    elif 'INTOX' in desc:
         new_desc.append('Drunk')
    elif 'TRAFFIC' in desc or 'VEHICLE' in desc:
         new_desc.append('Traffic')
    elif 'FORGERY' in desc or 'CRIMINAL MISCHIEF' in desc or 'MISC' in desc:
         new_desc.append('Other')
    elif 'SEX' in desc or 'RAPE':
         new_desc.append('SexCrime')
    else:
        new_desc.append(desc)
        
data['new_desc'] = new_desc
data['new_desc'].value_counts()

Theft       1744841
Assault     1409795
Other        673798
SexCrime     648643
Drugs        348495
Weapons      124235
Drunk         73778
Traffic       73072
Murder         4574
Name: new_desc, dtype: int64

In [98]:
data.to_csv('Data/partial_nypd_complaint_map.csv',index=False)

In [6]:
cols_311 = ['Agency','Incident Zip']
            #Complaint Type','Descriptor','Incident Zip','Latitude','Longitude']

In [7]:
data = pd.read_csv('Data/later_311_requests.csv',usecols=cols_311)
data.head()

,Agency,Agency Name,Incident Zip
0,DOB,Department of Buildings,11238
1,DOB,Department of Buildings,10466
2,DOB,Department of Buildings,10032
3,DOB,Department of Buildings,10012
4,DOB,Department of Buildings,10012


In [16]:
agency_list = ['HPD','NYPD','DOT','DEP','DSNY','DPR','DOB','DOF','DOHMH','TLC','DCA','DHS',
               'HRA','DFTA','FDNY','3-1-1','DOE','EDC','DOITT']
new_agency = []
for i in data.index:
    agency = data['Agency'][i]
    if agency in agency_list:
        new_agency.append(agency)
    else:
        new_agency.append('Other')
data['new_agency'] = new_agency
data['new_agency'].value_counts()

HPD      4210663
NYPD     2865895
DOT      2029494
DEP      1153687
DSNY      945141
DPR       647137
DOB       627241
DOF       531902
DOHMH     359997
TLC       163666
DCA       144698
DHS       124101
HRA        82142
DFTA       70680
FDNY       36284
3-1-1      24004
DOE        13753
EDC         6983
DOITT       6044
Other       5271
Name: new_agency, dtype: int64

In [17]:
data.to_csv('Data/partial_311_requests.csv',index=False)

In [24]:
data = pd.read_csv('Data/later_athletic_facilities.csv')
data.head()

,the_geom,SYSTEM,DEPARTMENT,GISPROPNUM,COMMUNITYB,COUNCILDIS,PRECINCT,ZIPCODE,BOROUGH,SubZnName,...,Hockey,TrackandFi,Lacrosse,Handball,Volleyball,Netball,Bocce,Rugby,FlagFootba,Wheelchair
0,MULTIPOLYGON (((-73.95787917040873 40.72902907...,B001-BASKETBALL-1,B-01,B001,301,33,94,11222,B,Basketball-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MULTIPOLYGON (((-73.95807261912218 40.72901274...,B001-BASKETBALL-2,B-01,B001,301,33,94,11222,B,Basketball-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MULTIPOLYGON (((-73.9582228973356 40.728763708...,B001-HANDBALL-1,B-01,B001,301,33,94,11222,B,Handball-02,...,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN
3,MULTIPOLYGON (((-73.95824361248151 40.72888286...,B001-HANDBALL-2,B-01,B001,301,33,94,11222,B,Handball-01,...,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN
4,MULTIPOLYGON (((-74.0014599836656 40.597099568...,B007-BASEBALL-1,B-11,B007,311,47,62,11214,B,Softball-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
data.columns.values

array(['the_geom', 'SYSTEM', 'DEPARTMENT', 'GISPROPNUM', 'COMMUNITYB',
       'COUNCILDIS', 'PRECINCT', 'ZIPCODE', 'BOROUGH', 'SubZnName',
       'DIMENSIONS', 'FIELD_LIGH', 'FIELD_NUMB', 'NAME', 'SURFACE_TY',
       'Zone_Name', 'Frisbee', 'AdultBaseb', 'AdultSoftb', 'AdultFootb',
       'Youthfootb', 'NonRegulat', 'LLBaseb13a', 'LLBaseb12a', 'TBall',
       'LLSoftball', 'Kickball', 'Cricket', 'Tennis', 'Basketball',
       'Hockey', 'TrackandFi', 'Lacrosse', 'Handball', 'Volleyball',
       'Netball', 'Bocce', 'Rugby', 'FlagFootba', 'Wheelchair'], dtype=object)

In [27]:
data['SYSTEM'].value_counts()

B073-ZN28-SOCCER-6      2
B073-ZN27-SOCCER-8      2
B073-ZN27-SOCCER-1      2
Q471-BASKETBALL-2       2
Q021-ZN06-CRICKET-1     2
B073-ZN27-SOCCER-2      2
B073-ZN28-SOCCER-11     2
B073-ZN27-SOCCER-7      2
Q471-BASKETBALL-1       2
B266-FOOTBALL-1         2
B073-ZN28-SOCCER-5      2
Q021-ZN06-SOCCER-6      2
B073-ZN28-SOCCER-10     2
B266-TRACK-1            2
X155-TRACK-1            1
B353-BASKETBALL-1       1
Q320-BASKETBALL-1       1
B163-HANDBALL-4         1
B353-BASKETBALL-2       1
X232-HANDBALL-2         1
X196-BASKETBALL-2       1
X196-BASKETBALL-3       1
B269-BASKETBALL-7       1
X196-BASKETBALL-1       1
X002-ZN01-HANDBALL-4    1
M010-ZN06-BASEBALL-5    1
Q024-03-HANDBALL-4      1
Q024-03-HANDBALL-3      1
Q024-03-HANDBALL-2      1
B321-BASKETBALL-3       1
                       ..
Q448-ZN03-TRACK-1       1
B271-HANDBALL-1         1
X047-BASEBALL-2         1
B271-HANDBALL-3         1
B271-HANDBALL-2         1
Q104-BASEBALL-3         1
M071-21-BASKETBALL-1    1
M071-21-BASK

In [33]:
new_sport = []
for i in data.index:
    sport = str(data['SYSTEM'][i])
    if 'SOCCER' in sport:
        new_sport.append('Soccer')
    elif 'BASKETBALL' in sport:
        new_sport.append('Basketball')
    elif 'HANDBALL' in sport:
        new_sport.append('Handball')
    elif 'TRACK' in sport:
        new_sport.append('Track')
    elif 'FOOTBALL' in sport:
        new_sport.append('Football')
    elif 'TENNIS' in sport:
        new_sport.append('Tennis')
    elif 'CRICKET' in sport or 'BOCCE':
        new_sport.append('Other')
    else:
        new_sport.append(sport)
data['new_sport'] = new_sport
data['new_sport'].value_counts()

Handball      2014
Basketball    1779
Other         1096
Tennis         698
Soccer         258
Football        75
Track           56
Name: new_sport, dtype: int64

In [34]:
data.to_csv('Data/partial_athletic_facilities.csv',index=False)

In [48]:
data = pd.read_csv('Data/later_businesses.csv')
data.head()

,DCA License Number,License Type,License Expiration Date,License Category,Business Name,Business Name 2,Address Building,Address Street Name,Secondary Address Street Name,Address City,Address State,Address ZIP,Contact Phone Number,Address Borough,Detail,Longitude,Latitude
0,2002435-DCA,Business,02/28/2017,Home Improvement Contractor,"TPC CONSTRUCTION, LLC",NaN,62,GRAND AVE,NaN,STATEN ISLAND,NY,10301,NaN,Staten Island,NaN,-74.10017,40.616465
1,1167184-DCA,Individual,10/31/2018,Tow Truck Driver,"HASSANEIN, HANY",NaN,2632,W 2ND ST,NaN,BROOKLYN,NY,11223,7187598994,NaN,NaN,NaN,NaN
2,1039848-DCA,Individual,09/30/2017,General Vendor,"WADE, CARRIE",NaN,350,W 24TH ST,NaN,NEW YORK,NY,10011,2126331568,NaN,NaN,NaN,NaN
3,2020172-DCA,Individual,04/30/2017,Pedicab Driver,"KALAJI, FIRAS HEKMAT",NaN,338,78TH ST,NaN,BROOKLYN,NY,11209,9174742164,NaN,NaN,NaN,NaN
4,2034537-DCA,Individual,05/31/2017,Locksmith,"RAMOS, TONY",NaN,159-44,HARLEM RIVER DR,NaN,NEW YORK,NY,10039,6469195163,NaN,NaN,NaN,NaN


In [49]:
data['License Category'].value_counts()

Home Improvement Salesperson      16215
Home Improvement Contractor       14509
Cigarette Retail Dealer            9001
Secondhand Dealer - General        5154
Electronics Store                  4434
Locksmith                          2645
Laundry                            2389
Stoop Line Stand                   2055
General Vendor                     2026
Tow Truck Driver                   1990
Electronic & Appliance Service     1686
Laundry Jobber                     1620
Debt Collection Agency             1560
Sidewalk Cafe                      1387
Garage                             1282
Ticket Seller                      1269
Pedicab Driver                      807
Secondhand Dealer - Auto            767
Dealer In Products                  762
Process Server Individual           758
Amusement Device Portable           585
Tow Truck Company                   551
Parking Lot                         454
Pawnbroker                          352
Auctioneer                          318


In [62]:
new_business = []
for i in data.index:
    business = str(data['License Category'][i])
    if 'Home' in business:
        new_business.append('Home Improvement')
    elif 'Retail' in business or 'Secondhand' in business or 'Store' in business or 'Stand' in business or 'Vendor' in business:
        new_business.append('Store')  
    elif 'Locksmith' in business or 'Laundry' in business or 'Tow' in business or 'Service' in business or 'Agency' in business:
        new_business.append('Service')    
    else:
        new_business.append('Other')
data['new_business'] = new_business
data['new_business'].value_counts()

Home Improvement    30724
Store               23443
Service             12921
Other                9770
Name: new_business, dtype: int64

In [63]:
data.to_csv('Data/partial_businesses.csv',index=False)

In [74]:
data = pd.read_csv('Data/later_collisions.csv')
data.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,11/10/2016,15:11,NaN,NaN,NaN,NaN,NaN,MONROE STREET,NaN,NaN,...,Unspecified,NaN,NaN,NaN,3559084,PASSENGER VEHICLE,NaN,NaN,NaN,NaN
1,11/10/2016,15:11,NaN,NaN,NaN,NaN,NaN,QUEENS PLAZA NORTH,NaN,NaN,...,Unspecified,NaN,NaN,NaN,3558161,PASSENGER VEHICLE,MOTORCYCLE,NaN,NaN,NaN
2,11/10/2016,15:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROADWAY,...,Unspecified,NaN,NaN,NaN,3559158,PASSENGER VEHICLE,NaN,NaN,NaN,NaN
3,11/10/2016,15:11,NaN,NaN,40.599075,-73.904973,"(40.599075, -73.9049732)",NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,3558441,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
4,11/10/2016,15:11,NaN,NaN,40.666310,-73.791699,"(40.6663098, -73.7916994)",NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,3557466,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN


(927517, 29)

In [77]:
data = data[np.isfinite(data['ZIP CODE'])]
data.shape

(689925, 29)

In [78]:
data.to_csv('Data/partial_collisions.csv',index=False)

In [79]:
data = pd.read_csv('Data/later_demographics_zipcode.csv')
data.head()

,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,10001,44,22,0.50,22,0.50,0,0,44,100,...,44,100,20,0.45,24,0.55,0,0,44,100
1,10002,35,19,0.54,16,0.46,0,0,35,100,...,35,100,2,0.06,33,0.94,0,0,35,100
2,10003,1,1,1.00,0,0.00,0,0,1,100,...,1,100,0,0.00,1,1.00,0,0,1,100
3,10004,0,0,0.00,0,0.00,0,0,0,0,...,0,0,0,0.00,0,0.00,0,0,0,0
4,10005,2,2,1.00,0,0.00,0,0,2,100,...,2,100,0,0.00,2,1.00,0,0,2,100


In [80]:
data.columns.values

array(['JURISDICTION NAME', 'COUNT PARTICIPANTS', 'COUNT FEMALE',
       'PERCENT FEMALE', 'COUNT MALE', 'PERCENT MALE',
       'COUNT GENDER UNKNOWN', 'PERCENT GENDER UNKNOWN',
       'COUNT GENDER TOTAL', 'PERCENT GENDER TOTAL',
       'COUNT PACIFIC ISLANDER', 'PERCENT PACIFIC ISLANDER',
       'COUNT HISPANIC LATINO', 'PERCENT HISPANIC LATINO',
       'COUNT AMERICAN INDIAN', 'PERCENT AMERICAN INDIAN',
       'COUNT ASIAN NON HISPANIC', 'PERCENT ASIAN NON HISPANIC',
       'COUNT WHITE NON HISPANIC', 'PERCENT WHITE NON HISPANIC',
       'COUNT BLACK NON HISPANIC', 'PERCENT BLACK NON HISPANIC',
       'COUNT OTHER ETHNICITY', 'PERCENT OTHER ETHNICITY',
       'COUNT ETHNICITY UNKNOWN', 'PERCENT ETHNICITY UNKNOWN',
       'COUNT ETHNICITY TOTAL', 'PERCENT ETHNICITY TOTAL',
       'COUNT PERMANENT RESIDENT ALIEN',
       'PERCENT PERMANENT RESIDENT ALIEN', 'COUNT US CITIZEN',
       'PERCENT US CITIZEN', 'COUNT OTHER CITIZEN STATUS',
       'PERCENT OTHER CITIZEN STATUS', 'COUNT CITIZ

In [81]:
data = pd.read_csv('Data/later_elevation.csv')
data.head()

,FEAT_CODE,ELEVATION,the_geom,SOURCE_ID,SUB_CODE,STATUS
0,3020,129.740000,POINT (-73.98256951739029 40.70191431011495),21302000001,302000,Unchanged
1,3000,120.586263,POINT (-73.98777990215136 40.70192587201795),21300000002,300020,Unchanged
2,3020,114.740000,POINT (-73.9844362285535 40.70193746867856),21302000003,302000,Unchanged
3,3020,69.130000,POINT (-73.98103106115082 40.70194154237234),21302000004,302000,Unchanged
4,3020,49.235397,POINT (-73.98802335685477 40.70196086988672),21302000005,302000,Unchanged


In [ ]:
data = fix_geom(data)

/Users/Laura/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Laura/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [87]:
for i in bbl_data.index[:20]:
    print i 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [6]:
data = pd.read_csv('Data/later_pedestrian_counts.csv')
data.head()

,the_geom,Loc,Borough,Street,From_,To,Index,May07_D,May07_AM,May07_PM,...,Sept14_D,Sept14_AM,Sept14_PM,Sept14_D2,Sept14_MD,May15_D,May15_AM,May15_PM,May15_D2,May15_MD
0,POINT (-73.90459140730687 40.879198966485745),1,Bronx,Broadway,West 231st Street,Naples Terrace,N,5/10/2007,1189,4094,...,09/04/14,1450,4646,09/06/14,2996,05/13/15,1788,4980,05/16/15,3033
1,POINT (-73.92188432870225 40.82662794123293),2,Bronx,East 161st Street,Grand Concourse,Sheridan Avenue,Y,5/10/2007,1511,3184,...,09/11/14,1949,4435,09/13/14,2388,05/19/15,2318,4589,05/16/15,2483
2,POINT (-73.89535781584343 40.862154600315165),3,Bronx,East Fordham Road,Valentine Avenue,Tiebout Avenue,Y,5/10/2007,1832,12311,...,09/04/14,2752,10408,09/06/14,8266,05/13/15,2382,11216,05/16/15,7574
3,POINT (-73.87892467324485 40.881286995987296),4,Bronx,East Gun Hill Road,Bainbridge Avenue,Rochambeau Avenue,N,5/10/2007,764,2673,...,09/04/14,2382,3425,09/06/14,1339,05/13/15,1757,3561,05/16/15,1714
4,POINT (-73.88956389732796 40.84463677671767),5,Bronx,East Tremont Avenue,Prospect Avenue,Clinton Avenue,N,5/10/2007,650,2538,...,09/04/14,1019,3184,09/06/14,2366,05/21/15,1055,3710,05/09/15,2554


In [9]:
data = fix_geom(data)

/Users/Laura/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Laura/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
data.to_csv('Data/partial_pedestrian_counts.csv',index=False)

In [24]:
data = pd.read_csv('Data/later_subways.csv')
data = fix_geom(data)
data.head()

/Users/Laura/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Laura/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,NAME,the_geom,URL,LINE,latitude,longitude
0,Smith St & Bergen St At Ne Corner (To Manhatta...,POINT (-73.99027199929638 40.686728331297715),http://www.mta.info/nyct/service/,F-G,40.686728,-73.990272
1,Court St & Montague St At Sw Corner,POINT (-73.9906780008789 40.69372533216928),http://www.mta.info/nyct/service/,2-3-4-5-N-R,40.693725,-73.990678
2,Court St & Montague St At Sw Corner,POINT (-73.99059199905471 40.693642331621014),http://www.mta.info/nyct/service/,2-3-4-5-N-R,40.693642,-73.990592
3,Clinton St & Montague St At Nw Corner,POINT (-73.99253735666481 40.69439311951264),http://www.mta.info/nyct/service/,2-3-4-5-N-R,40.694393,-73.992537
4,Flatbush Ave & Empire Blvd At Sw Corner,POINT (-73.96224891505983 40.66272735822238),http://www.mta.info/nyct/service/,B-Q-S,40.662727,-73.962249


In [26]:
data.to_csv('Data/partial_subways.csv',index=False)

In [33]:
data = pd.read_csv('Data/partial_311_requests.csv')

/Users/Laura/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
data.head()

,Agency,Agency Name,Incident Zip,new_agency
0,DOB,Department of Buildings,11238,DOB
1,DOB,Department of Buildings,10466,DOB
2,DOB,Department of Buildings,10032,DOB
3,DOB,Department of Buildings,10012,DOB
4,DOB,Department of Buildings,10012,DOB


In [30]:
data.columns.values

array(['FID', 'tree_id', 'block_id', 'created_at', 'tree_dbh',
       'stump_diam', 'curb_loc', 'status', 'health', 'spc_latin',
       'spc_common', 'steward', 'guards', 'sidewalk', 'user_type',
       'root_stone', 'root_grate', 'root_other', 'trunk_wire',
       'trnk_light', 'trnk_other', 'brch_light', 'brch_shoe', 'brch_other',
       'address', 'zipcode', 'zip_city', 'cb_num', 'borocode', 'boroname',
       'cncldist', 'st_assem', 'st_senate', 'nta', 'nta_name', 'boro_ct',
       'state', 'latitude', 'longitude', 'x_sp', 'y_sp'], dtype=object)

In [ ]:
def fix_MULTIPOLYGON_geom(data):
    data['latitude'] = np.nan * data.index
    data['longitude'] = np.nan * data.index
    for i in data.index:
        try:
            temp = data['the_geom'][i]
            temp = temp.replace(',', '')
            temp = str.strip(temp, 'MULTIPOLYGON (((')
            temp = str.strip(temp, ')))')
            temp = map(float, temp.split())
            temp = np.asarray(temp).reshape(len(temp)/2,2)
            temp = LinearRing(temp)
            temp = temp.representative_point().wkt
            temp = str.strip(temp, 'POINT (')
            temp = str.strip(temp, ')')
            temp = temp.split()
            data['latitude'][i] = temp[1]
            data['longitude'][i] = temp[0]
        except:
            data['latitude'][i] = np.nan
            data['longitude'][i] = np.nan
    data = data.dropna(axis=0)
    return data